In [ ]:
import os
import pandas as pd
import pymongo
import yaml
from datetime import datetime
from utils.data_processing import df_rebase

# Load configuration
config_path = os.path.join(os.getcwd(), "..", "config.yml")
with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Connect to MongoDB
client = pymongo.MongoClient(config["client"])
db = client[config["db"]]
col = db[config["col"]]

# Load dataset and upload to MongoDB
data_path = os.path.join(os.getcwd(), "..", config["data_path"])

def upload_data(data_path, col):
    classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
    for class_folder in classes_folders_list:
        class_folder_path = os.path.join(data_path, class_folder)
        files_in_folder = [f for f in os.listdir(class_folder_path) if os.path.isfile(os.path.join(class_folder_path, f))]
        for file in files_in_folder:
            file_path = os.path.join(class_folder_path, file)
            df = pd.read_csv(file_path)
            df = df_rebase(df, config['order'], config['rename'])
            document = {
                "data": df.to_dict('list'),
                "label": class_folder,
                "datetime": datetime.now()
            }
            col.insert_one(document)

upload_data(data_path, col)
